# Initial Data Preparation 1.0
done by Jack Phelan

This notebook covers the initial data exploration and cleaning steps for the apnea predictor project. The goal is to understand the dataset, identify any missing or inconsistent values, and prepare the data for further analysis and modeling.

In [245]:
# imports

import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# custom module imports

folder_path = os.path.abspath(os.path.join(os.getcwd(), "..", "utils"))
if folder_path not in sys.path:
    sys.path.insert(0, folder_path)



import data_utils as du


# function import


importlib.reload(du)  # while developing the module

<module 'data_utils' from '/Users/jack/Repos/apnea-predictor/utils/data_utils.py'>

In [246]:
# loading
stages_raw_df = pd.read_csv("../data/raw/stages/datasets/stages-dataset-0.3.0.csv")
harmonized_stages_raw_df = pd.read_csv(
    "../data/raw/stages/datasets/stages-harmonized-dataset-0.3.0.csv"
)


# preview
display(stages_raw_df.head())
display(harmonized_stages_raw_df.head())

,modified_created_at,modified_completed,subject_code,dem_0100,modified_dem_0110,dem_0500,dem_0600,dem_0610,dem_0700,dem_0800,...,sched_1701,sched_1801,soclhx_0101,narc_1710,never_cigarette_smoker,former_cigarette_smoker,former_smokeless_user,current_cigarette_smoker,current_smokeless_user,visitcode
0,28SEP18:17:25:32,NaN,BOGN00002,1960.0,58.0,F,5.0,2.0,168.0,30.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,12OCT18:08:07:19,12OCT18:08:48:00,BOGN00004,1987.0,30.0,F,5.0,7.0,188.0,29.4,...,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1
2,08MAR19:07:35:09,08MAR19:07:55:00,BOGN00007,1988.0,30.0,F,5.0,7.0,165.0,25.8,...,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1
3,18OCT18:16:06:54,18OCT18:16:43:00,BOGN00008,1976.0,42.0,M,5.0,4.0,156.0,26.8,...,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1
4,01MAR19:11:55:20,01MAR19:12:23:00,BOGN00009,1982.0,36.0,M,5.0,3.0,255.0,45.2,...,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1


,subject_code,visitcode,nsrr_age,nsrr_age_gt89,nsrr_sex,nsrr_race,nsrr_ethnicity,nsrr_bmi,nsrr_current_smoker,nsrr_ever_smoker
0,BOGN00002,1,58.0,no,female,white,not hispanic or latino,30.7,not reported,not reported
1,BOGN00004,1,30.0,no,female,white,not hispanic or latino,29.4,no,no
2,BOGN00007,1,30.0,no,female,white,not hispanic or latino,25.8,no,no
3,BOGN00008,1,42.0,no,male,white,not hispanic or latino,26.8,no,yes
4,BOGN00009,1,36.0,no,male,white,not hispanic or latino,45.2,no,yes


In [247]:
# info
display(stages_raw_df.info())
display(harmonized_stages_raw_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Columns: 433 entries, modified_created_at to visitcode
dtypes: float64(402), int64(1), object(30)
memory usage: 6.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   subject_code         1881 non-null   object 
 1   visitcode            1881 non-null   int64  
 2   nsrr_age             1859 non-null   float64
 3   nsrr_age_gt89        1881 non-null   object 
 4   nsrr_sex             1859 non-null   object 
 5   nsrr_race            1859 non-null   object 
 6   nsrr_ethnicity       1881 non-null   object 
 7   nsrr_bmi             1859 non-null   float64
 8   nsrr_current_smoker  1881 non-null   object 
 9   nsrr_ever_smoker     1881 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 147.1+ KB


None

Since column names are encoded, I will load the data dictionary and use a multi index 
for the columns to make exploration easier.

In [248]:
# data dictionary adding
data_dict = pd.read_csv(
    "../data/raw/stages/datasets/stages-data-dictionary-0.3.0-variables.csv"
)
display(data_dict.info())
display_names = data_dict.set_index("id")["display_name"].to_dict()

# make strings easier to work with
stripped_names = du.strip_text(display_names.values())

keys = list(display_names.keys())  # getting associated columns
stripped_names_dict = {keys[i]: stripped_names[i] for i in range(len(keys))}  # new dict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   folder         441 non-null    object 
 1   id             441 non-null    object 
 2   display_name   441 non-null    object 
 3   description    365 non-null    object 
 4   type           441 non-null    object 
 5   units          107 non-null    object 
 6   domain         301 non-null    object 
 7   labels         295 non-null    object 
 8   calculation    24 non-null     object 
 9   commonly_used  15 non-null     object 
 10  forms          0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 38.0+ KB


None

In [249]:
# adding multi index to dataframe
stages_mi_df = du.add_multi_index(stages_raw_df, stripped_names_dict)

stages_mi_df.head()


date_asq_created date_asq_completed stages_subject_identifier  \
  modified_created_at modified_completed              subject_code   
0    28SEP18:17:25:32                NaN                 BOGN00002   
1    12OCT18:08:07:19   12OCT18:08:48:00                 BOGN00004   
2    08MAR19:07:35:09   08MAR19:07:55:00                 BOGN00007   
3    18OCT18:16:06:54   18OCT18:16:43:00                 BOGN00008   
4    01MAR19:11:55:20   01MAR19:12:23:00                 BOGN00009   

  participants_year_of_birth  participants_age participants_sex  \
                    dem_0100 modified_dem_0110         dem_0500   
0                     1960.0              58.0                F   
1                     1987.0              30.0                F   
2                     1988.0              30.0                F   
3                     1976.0              42.0                M   
4                     1982.0              36.0                M   

  height_in_feet height_in_inches weight_in_pounds body_mass_index_(bmi)  ...  \
        dem_0600         dem_0610         dem_0700              dem_0800  ...   
0            5.0              2.0            168.0                  30.7  ...   
1            5.0              7.0            188.0                  29.4  ...   
2            5.0              7.0            165.0                  25.8  ...   
3            5.0              4.0            156.0                  26.8  ...   
4            5.0              3.0            255.0                  45.2  ...   

  self-reported_in-bed_time_on_week_nights_school_nights_work_nights_or_days_3rd_shift_varies  \
                                                                                   sched_1701   
0                                                NaN                                            
1                                                NaN                                            
2                                                NaN                                            
3                                                NaN                                            
4                                                NaN                                            

             rarely_or_never_nap muscle_weakness_month_of_the_first_episode  \
  sched_1801         soclhx_0101                                  narc_1710   
0        NaN                 NaN                                        NaN   
1        NaN                 NaN                                        0.0   
2        NaN                 NaN                                        0.0   
3        NaN                 NaN                                        0.0   
4        NaN                 NaN                                        0.0   

  cigarette_smoking_never_smoker cigarette_smoking_former_smoker  \
          never_cigarette_smoker         former_cigarette_smoker   
0                            NaN                             NaN   
1                            1.0                             0.0   
2                            1.0                             0.0   
3                            0.0                             1.0   
4                            0.0                             1.0   

  smokeless_user_former_smoker cigarette_smoking_current_smoker  \
         former_smokeless_user         current_cigarette_smoker   
0                          NaN                              NaN   
1                          0.0                              0.0   
2                          0.0                              0.0   
3                          0.0                              0.0   
4                          0.0                              0.0   

  smokeless_user_current_smoker stages_visit  
         current_smokeless_user    visitcode  
0                           NaN            1  
1                           0.0            1  
2                           0.0            1  
3                           0.0            1  
4            

## Multi Index Usage
dataframe is now easier to understand and contains a multiindex; 

- index level 0 is the descriptive name from the data dictionary
- index level 1 is the original column name

the harmonized dataset is small and descriptive enough that multiindexing is not necessary

In [250]:
stages_mi_df.describe()

participants_year_of_birth  participants_age height_in_feet  \
                        dem_0100 modified_dem_0110       dem_0600   
count                1859.000000       1859.000000    1859.000000   
mean                 1972.201183         45.889188       5.144164   
std                    15.185022         15.202918       0.404072   
min                  1934.000000         13.000000       4.000000   
25%                  1960.000000         33.500000       5.000000   
50%                  1972.000000         46.000000       5.000000   
75%                  1985.000000         58.000000       5.000000   
max                  2006.000000         84.000000       7.000000   

      height_in_inches weight_in_pounds body_mass_index_(bmi)  \
              dem_0610         dem_0700              dem_0800   
count      1859.000000      1859.000000           1859.000000   
mean          5.318989       200.433566             31.315223   
std           3.341670        60.231383              8.902121   
min           0.000000        73.000000             11.900000   
25%           2.000000       160.000000             25.100000   
50%           5.000000       190.000000             29.300000   
75%           8.000000       230.000000             35.700000   
max          11.000000       546.000000             79.500000   

      participants_ethnicity_(hispanic_or_latino)  \
                                         dem_0900   
count                                 1859.000000   
mean                                     0.061861   
std                                      0.240968   
min                                      0.000000   
25%                                      0.000000   
50%                                      0.000000   
75%                                      0.000000   
max                                      1.000000   

      participants_ethnicity_(sub_hispanic_or_latino_origin)  \
                                                    dem_0910   
count                                         109.000000       
mean                                            1.357798       
std                                             1.397901       
min                                             0.000000       
25%                                             0.000000       
50%                                             1.000000       
75%                                             3.000000       
max                                             3.000000       

      participants_race_(main) participants_race_(sub)  ...  \
                      dem_1000                dem_1010  ...   
count              1859.000000             1755.000000  ...   
mean                  1.665949                1.141311  ...   
std                   1.457610                2.105668  ...   
min                   1.000000                0.000000  ...   
25%                   1.000000                0.000000  ...   
50%                   1.000000                0.000000  ...   
75%                   1.000000                1.000000  ...   
max                   6.000000               15.000000  ...   

      self-reported_in-bed_time_on_week_nights_school_nights_work_nights_or_days_3rd_shift_varies  \
                                                                                       sched_1701   
count                                               16.0                                            
mean                                                 1.0                                            
std                                                  0.0                                            
min                                                  1.0                                            
25%                                                  1.0                                            
50%                                                  1.0                                            
75%                                                  

## Subsetting and Sorting Columns
to make summary statistics easier to understand and the data easier to work with, I will

use folder and label  information from the data dictionary to subset and sort columns

in the main dataframe.

In [251]:
# Subsetting and Sorting Columns

folders_and_labels = data_dict[["folder", "labels", "id"]]
folders_and_labels = folders_and_labels.set_index("id")
folders_and_labels_dict = folders_and_labels.to_dict(orient="index")

display(folders_and_labels['folder'].value_counts())
display(folders_and_labels['labels'].value_counts())


folder
Sleep Questionnaires/Hypersomnia                                                             72
Sleep Questionnaires/Sleep Disturbance                                                       63
Lifestyle and Behavioral Health                                                              53
Sleep Questionnaires/Sleep Habits                                                            41
Sleep Questionnaires/Parasomnia                                                              29
Medical History                                                                              24
Sleep Treatment/Adherence/CPAP Adherence                                                     18
Medical History/Family History                                                               13
Sleep Questionnaires/Sleep Disturbance/Functional Outcomes of Sleep Questionnaire (FOSQ)     11
Demographics/Employment                                                                      11
General Health/Patient Health Que

labels
narcolepsy                   39
insomnia                     36
rls_plms                     18
work_schedule                16
sleep_timing                 16
                             ..
current_smoker_baseline       1
annotated_sex                 1
race                          1
ethnicity                     1
epworth;ess_1sitread          1
Name: count, Length: 88, dtype: int64

Subsetting thoughts first iteration
- use labels first and see the breakdown of the rest of the columns

In [252]:
# grouping columns by labels (with >1 count)


# Group by labels and see the structure
grouped_by_labels = folders_and_labels.groupby('labels')

# To see each group
for label, group in grouped_by_labels:
    if len(group) > 1:
        print(f"Label: {label}")
        print(group)
        print()

Label: bruxism
                                    folder   labels
id                                                 
par_0400  Sleep Questionnaires/Hypersomnia  bruxism
par_0500  Sleep Questionnaires/Hypersomnia  bruxism
par_0501  Sleep Questionnaires/Hypersomnia  bruxism

Label: hypnogogic_hallucinations
                                   folder                     labels
id                                                                  
map_1100  Sleep Questionnaires/Parasomnia  hypnogogic_hallucinations
map_1110  Sleep Questionnaires/Parasomnia  hypnogogic_hallucinations
map_1120  Sleep Questionnaires/Parasomnia  hypnogogic_hallucinations
map_1130  Sleep Questionnaires/Parasomnia  hypnogogic_hallucinations
map_1131  Sleep Questionnaires/Parasomnia  hypnogogic_hallucinations

Label: insomnia
                                            folder    labels
id                                                          
nose_0400                           General Health  insomnia
isq_0100

In [253]:
# check ids that dont have lables

if folders_and_labels["labels"].isnull().any():
    no_label_ids = folders_and_labels[folders_and_labels["labels"].isnull()]

no_label_ids_folders = no_label_ids.groupby("folder")
for folder, group in no_label_ids_folders:
    print(f"Folder: {folder}")
    print(group)
    print()

Folder: Administrative
                             folder labels
id                                        
modified_completed   Administrative    NaN
modified_created_at  Administrative    NaN
subject_code         Administrative    NaN
visitcode            Administrative    NaN

Folder: Anthropometry
                 folder labels
id                            
dem_0600  Anthropometry    NaN
dem_0610  Anthropometry    NaN
dem_0700  Anthropometry    NaN
dem_0800  Anthropometry    NaN

Folder: Demographics
                         folder labels
id                                    
dem_0100           Demographics    NaN
dem_0500           Demographics    NaN
dem_0900           Demographics    NaN
dem_0910           Demographics    NaN
dem_1000           Demographics    NaN
dem_1010           Demographics    NaN
modified_dem_0110  Demographics    NaN

Folder: Demographics/Acculturation
                              folder labels
id                                         
dem_1100  Dem

I am going to add a 'group' and 'subgroup' column to the eventual dataframe that can then be used for subsetting and grouping data for analysis and modeling.

curent groups from folders:
- demographics (subgroups anthroprometrics, info, education, household_makeup)
- general_health (subgroups general_health,lifestyle and behavioral health)
- medical_history(subgroups personal history, family history, reproductive health)

current groups from labels:
- work_schedule
- subject_sleepiness
- sleepwalk
- sleep_timing
- sleep_satisfaction
- sleep_related_impairment
- sleep_quality
- sleep_paralysis
- sleep_latency
- sleep_hygiene
- sleep_eat
- sleep_duration
- sdb
- rls (restless leg syndrome)
- nightmare
- narcolepsy
- insomnia
- hypnogogic_hallucinations and bruxism

grouping
- hypnogoic_hallucinations, bruxism, sleep_paralysis, nightmare, sleepwalk -> parasomnias
- sleep_duration, sleep_latency, sleep_quality, sleep_satisfaction, sleep_timing,
 sleep_hygiene, sleep_eat, subject_sleepiness -> sleep_patterns
- 

In [254]:
# grouping columns from labels


parasomnias = folders_and_labels.where(
    (folders_and_labels["labels"] == "hypnogogic_hallucinations")
    | (folders_and_labels["labels"] == "bruxism")
    | (folders_and_labels["labels"] == "sleep_paralysis")
    | (folders_and_labels["labels"] == "nightmare")
    | (folders_and_labels["labels"] == "sleepwalk")
)

parasomnias.dropna(inplace=True, subset=["labels"])
parasomnias_columns = parasomnias.index.tolist()

sleep_patterns = folders_and_labels.where(
    (folders_and_labels["labels"] == "sleep_latency")
    | (folders_and_labels["labels"] == "sleep_quality")
    | (folders_and_labels["labels"] == "sleep_satisfaction")
    | (folders_and_labels["labels"] == "sleep_duration")
    | (folders_and_labels["labels"] == "sleep_timing")
    | (folders_and_labels["labels"] == "sleep_hygiene")
    | (folders_and_labels["labels"] == "sleep_eat")
    | (folders_and_labels["labels"] == "subject_sleepiness")
)
sleep_patterns.dropna(inplace=True, subset=["labels"])

sleep_patterns_columns = sleep_patterns.index.tolist()

rls_columns = folders_and_labels.where(folders_and_labels["labels"] == "rls_plms")
rls_columns.dropna(inplace=True, subset=["labels"])
rls_columns = rls_columns.index.tolist()

narc_columns = folders_and_labels.where(folders_and_labels["labels"] == "narcolepsy")
narc_columns.dropna(inplace=True, subset=["labels"])
narc_columns = narc_columns.index.tolist()

insomnia_columns = folders_and_labels.where(folders_and_labels["labels"] == "insomnia")
insomnia_columns.dropna(inplace=True, subset=["labels"])
insomnia_columns = insomnia_columns.index.tolist()


# grouping columns from folders


demographics_mask = folders_and_labels["folder"].str.startswith(
    "Demographics", na=False
)
anthropometry_mask = folders_and_labels["folder"].str.startswith(
    "Anthropometry", na=False
)

# Use in filtering
demographics = folders_and_labels[demographics_mask]
anthropometry = folders_and_labels[anthropometry_mask]

demographics = pd.concat([demographics, anthropometry], ignore_index=False)

demographics_columns = demographics.index.tolist()

health_mask = folders_and_labels["folder"].str.contains("Health", na=False)
health = folders_and_labels[health_mask]
health_columns = health.index.tolist()

medhx_mask = folders_and_labels["folder"].str.startswith("Medical History", na=False)

medhx = folders_and_labels[medhx_mask]
medhx_columns = medhx.index.tolist()


sleep_questions_mask = folders_and_labels["folder"].str.startswith(
    "Sleep Questionnaires", na=False
)
sleep_questions = folders_and_labels[sleep_questions_mask]
sleep_questions_columns = sleep_questions.index.tolist()

sleep_treatment_mask = folders_and_labels["folder"].str.startswith(
    "Sleep Treatment", na=False
)
sleep_treatment = folders_and_labels[sleep_treatment_mask]
sleep_treatment_columns = sleep_treatment.index.tolist()

In [255]:
# checking if any columns are missed

all_selected_columns = (
    parasomnias_columns
    + sleep_patterns_columns
    + demographics_columns
    + health_columns
    + medhx_columns
    + rls_columns
    + narc_columns
    + insomnia_columns
    + sleep_questions_columns
    + sleep_treatment_columns
)

folders_and_labels_ids = folders_and_labels.index.tolist()
missed_columns = set(folders_and_labels_ids) - set(all_selected_columns)

missed = folders_and_labels.loc[list(missed_columns)]
display(missed)

,folder,labels
id,,
nsrr_sex,Harmonized/Demographics,annotated_sex
modified_completed,Administrative,NaN
nsrr_bmi,Harmonized/Anthropometry,bmi_baseline_1
modified_created_at,Administrative,NaN
visitcode,Administrative,NaN
nsrr_ethnicity,Harmonized/Demographics,ethnicity
subject_code,Administrative,NaN
nsrr_age,Harmonized/Demographics,age_at_index
nsrr_age_gt89,Harmonized/Demographics,age_at_index_gt89


In [256]:
# check for duplicates in the subsets

setlist = [
    set(parasomnias_columns),
    set(sleep_patterns_columns),
    set(demographics_columns),
    set(health_columns),
    set(medhx_columns),
    set(rls_columns),
    set(narc_columns),
    set(insomnia_columns),
    set(sleep_questions_columns),
    set(sleep_treatment_columns),
]

# Names for each subset (for easier identification)
subset_names = [
    "parasomnias",
    "sleep_patterns",
    "demographics",
    "health",
    "medhx",
    "rls",
    "narc",
    "insomnia",
    "sleep_questions",
    "sleep_treatment",
]
# storing overlaps
overlaps = {}


# Check for overlaps between all pairs
print("Checking for overlaps between subsets:")
for i in range(len(setlist)):
    for j in range(i + 1, len(setlist)):
        overlap = setlist[i].intersection(setlist[j])
        if overlap:
            print(f"{subset_names[i]} & {subset_names[j]}: {overlap}")
            overlaps[(subset_names[i], subset_names[j])] = overlap

# Check if any column appears in multiple subsets
all_columns = []
for subset in setlist:
    all_columns.extend(list(subset))

from collections import Counter

column_counts = Counter(all_columns)
duplicates = {col: count for col, count in column_counts.items() if count > 1}

print(f"\nColumns appearing in multiple subsets: {duplicates}")

Checking for overlaps between subsets:
parasomnias & sleep_questions: {'par_0800', 'par_0530', 'par_0601', 'par_0600', 'par_0110', 'par_0701', 'par_0230', 'map_1131', 'par_0610', 'par_0101', 'par_0210', 'par_0531', 'par_0510', 'map_1110', 'par_0500', 'map_1130', 'par_0201', 'par_0710', 'par_0501', 'map_1100', 'par_0630', 'par_0200', 'par_0700', 'par_0400', 'par_0100', 'map_1120', 'par_0631'}
sleep_patterns & health: {'soclhx_0900', 'soclhx_0901', 'soclhx_1000'}
sleep_patterns & sleep_questions: {'soclhx_0101', 'sched_4200', 'sched_3200', 'sched_3800', 'sched_2700', 'sched_1001', 'sched_1401', 'sched_2000', 'sched_1900', 'sched_3310', 'isq_0600', 'soclhx_0210', 'sched_4201', 'soclhx_0100', 'bthbts_0300', 'sched_2600', 'sched_4100', 'sched_1901', 'sched_2800', 'soclhx_0110', 'sched_1300', 'sched_4210', 'sched_1700', 'sched_1301', 'sched_1800', 'isq_0400', 'sched_2710', 'sched_0901', 'sched_2310', 'sched_3210', 'sched_3810', 'soclhx_0400', 'sched_2210', 'sched_3100', 'sched_2300', 'sched_

In [257]:
# make function to check for duplicates

def find_overlaps(sets, set_names):
    overlaps = {}
    for i in range(len(sets)):
        for j in range(i + 1, len(sets)):
            overlap = sets[i].intersection(sets[j])
            if overlap:
                overlaps[(set_names[i], set_names[j])] = overlap
    return overlaps



The only columns not accounted for are from the harmonized dataset (already present in
the main sheet) and the administrative codes that mark when the data was collected. 
however there are many overlapping columns that need to be handled in the subsets

In [258]:
# checking overlaps in stages_mi_df


# Get descriptive names for all duplicate columns at once
duplicate_descriptive_names = [
    (
        col,
        stages_mi_df.columns.get_level_values(0)[
            stages_mi_df.columns.get_level_values(1).get_loc(col)
        ],
    )
    for col in duplicates.keys()
    if col in stages_mi_df.columns.get_level_values(1)
]

print("Duplicate columns with descriptive names:")
for orig, desc in duplicate_descriptive_names:
    print(f"  {orig} -> {desc}")

Duplicate columns with descriptive names:
  par_0800 -> seizures_during_sleep
  par_0530 -> acting_out_dreams_age_of_the_first_episode
  par_0601 -> frequency_of_violent_behavior_during_sleep_never_or_dont_know
  par_0600 -> frequency_of_violent_behavior_during_sleep_times
  par_0110 -> frequency_of_leg_twitch/kick_time_frame
  par_0701 -> frequency_of_nightmares_never_or_dont_know
  par_0230 -> sleepwalk_age_of_the_first_episode
  map_1131 -> hypnogogic_hallucinations_age_of_the_first_episode_dont_know
  par_0610 -> frequency_of_violent_behavior_during_sleep_time_frame
  par_0101 -> frequency_of_leg_twitch/kick_never_or_dont_know
  par_0210 -> frequency_of_sleepwalk_time_frame
  par_0531 -> acting_out_dreams_age_of_the_first_episode_dont_know
  par_0510 -> frequency_of_acting_out_dreams_time_frame
  map_1110 -> hypnogogic_hallucinations_number_of_times
  par_0500 -> frequency_of_acting_out_dreams_times
  map_1130 -> hypnogogic_hallucinations_age_of_the_first_episode
  par_0201 -> freq

In [259]:
# dropping dupes from subsets

sleep_questions_columns = [
    col for col in sleep_questions_columns if col not in parasomnias_columns
]

sleep_patterns_columns = [
    col for col in sleep_patterns_columns if col not in health_columns
]

sleep_questions_columns = [
    col for col in sleep_questions_columns if col not in sleep_patterns_columns
]

health_columns = [
    col for col in health_columns if col not in medhx_columns
]

insomnia_columns = [
    col for col in insomnia_columns if col not in health_columns
]

sleep_questions_columns = [
    col for col in sleep_questions_columns if col not in rls_columns
]

sleep_questions_columns = [
    col for col in sleep_questions_columns if col not in narc_columns
]

sleep_questions_columns = [
    col for col in sleep_questions_columns if col not in insomnia_columns
]




In [260]:
# recheck dupes

setlist = [
    set(parasomnias_columns),
    set(sleep_patterns_columns),
    set(demographics_columns),
    set(health_columns),
    set(medhx_columns),
    set(rls_columns),
    set(narc_columns),
    set(insomnia_columns),
    set(sleep_questions_columns),
    set(sleep_treatment_columns),
]   

overlaps = find_overlaps(setlist, subset_names)
print("Overlaps after dropping duplicates:")
for (set1, set2), overlap in overlaps.items():
    print(f"{set1} & {set2}: {overlap}")

Overlaps after dropping duplicates:


In [261]:
print("\nNumber of features in each subset after dropping duplicates:")
for sets in setlist:
    print(subset_names[setlist.index(sets)]+':', len(sets))



Number of features in each subset after dropping duplicates:
parasomnias: 27
sleep_patterns: 58
demographics: 35
health: 92
medhx: 40
rls: 18
narc: 39
insomnia: 35
sleep_questions: 68
sleep_treatment: 19


In [262]:
# Create subset labels for each column
subset_labels = []
for col in stages_mi_df.columns.get_level_values(1):
    if col in parasomnias_columns:
        subset_labels.append('parasomnias')
    elif col in sleep_patterns_columns:
        subset_labels.append('sleep_patterns')
    elif col in demographics_columns:
        subset_labels.append('demographics')
    elif col in health_columns:
        subset_labels.append('health')
    elif col in medhx_columns:
        subset_labels.append('medhx')
    elif col in rls_columns:
        subset_labels.append('rls')
    elif col in narc_columns:
        subset_labels.append('narc')
    elif col in insomnia_columns:
        subset_labels.append('insomnia')
    elif col in sleep_questions_columns:
        subset_labels.append('sleep_questions')
    elif col in sleep_treatment_columns:
        subset_labels.append('sleep_treatment')
    else:
        subset_labels.append('unassigned')

# Create new MultiIndex with 3 levels: descriptive_name, original_name, subset
new_columns = pd.MultiIndex.from_arrays([
    stages_mi_df.columns.get_level_values(0),  # descriptive names
    stages_mi_df.columns.get_level_values(1),  # original names  
    subset_labels                               # subset labels
], names=['descriptive', 'original', 'subset'])

stages_mi_sub_df = stages_mi_df.copy()
stages_mi_sub_df.columns = new_columns

stages_mi_sub_df.head()

descriptive    date_asq_created date_asq_completed stages_subject_identifier  \
original    modified_created_at modified_completed              subject_code   
subset               unassigned         unassigned                unassigned   
0              28SEP18:17:25:32                NaN                 BOGN00002   
1              12OCT18:08:07:19   12OCT18:08:48:00                 BOGN00004   
2              08MAR19:07:35:09   08MAR19:07:55:00                 BOGN00007   
3              18OCT18:16:06:54   18OCT18:16:43:00                 BOGN00008   
4              01MAR19:11:55:20   01MAR19:12:23:00                 BOGN00009   

descriptive participants_year_of_birth  participants_age participants_sex  \
original                      dem_0100 modified_dem_0110         dem_0500   
subset                    demographics      demographics     demographics   
0                               1960.0              58.0                F   
1                               1987.0              30.0                F   
2                               1988.0              30.0                F   
3                               1976.0              42.0                M   
4                               1982.0              36.0                M   

descriptive height_in_feet height_in_inches weight_in_pounds  \
original          dem_0600         dem_0610         dem_0700   
subset        demographics     demographics     demographics   
0                      5.0              2.0            168.0   
1                      5.0              7.0            188.0   
2                      5.0              7.0            165.0   
3                      5.0              4.0            156.0   
4                      5.0              3.0            255.0   

descriptive body_mass_index_(bmi)  ...  \
original                 dem_0800  ...   
subset               demographics  ...   
0                            30.7  ...   
1                            29.4  ...   
2                            25.8  ...   
3                            26.8  ...   
4                            45.2  ...   

descriptive self-reported_in-bed_time_on_week_nights_school_nights_work_nights_or_days_3rd_shift_varies  \
original                                                                                     sched_1701   
subset                                                                                   sleep_patterns   
0                                                          NaN                                            
1                                                          NaN                                            
2                                                          NaN                                            
3                                                          NaN                                            
4                                                          NaN                                            

descriptive                rarely_or_never_nap  \
original        sched_1801         soclhx_0101   
subset      sleep_patterns      sleep_patterns   
0                      NaN                 NaN   
1                      NaN                 NaN   
2                      NaN                 NaN   
3                      NaN                 NaN   
4                      NaN                 NaN   

descriptive muscle_weakness_month_of_the_first_episode  \
original                                     narc_1710   
subset                                            narc   
0                                                  NaN   
1                                                  0.0   
2                                                  0.0   
3                                                  0.0   
4                                                  0.0   

descriptive cigarette_smoking_never_smoker cigarette_smoking_former_smoker  \
original            never_cigarette_smoker         former_cigarette_smoker   
subset       

## Quality Assessment
I will check for missing values, inconsistent data types, and outliers in the dataset. 
At first i will check missing and duplicates in the entire dataset, and then evaluate
each subset individually.

In [263]:
stages_mi_sub_df = stages_mi_sub_df.drop('unassigned', axis=1, level='subset')

stages_mi_sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Columns: 429 entries, ('participants_year_of_birth', 'dem_0100', 'demographics') to ('smokeless_user_current_smoker', 'current_smokeless_user', 'health')
dtypes: float64(402), object(27)
memory usage: 6.2+ MB


In [264]:
stages_mi_sub_df = stages_mi_sub_df.drop_duplicates()
stages_mi_sub_df = stages_mi_sub_df.dropna(how='all')

stages_mi_sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1860 entries, 0 to 1880
Columns: 429 entries, ('participants_year_of_birth', 'dem_0100', 'demographics') to ('smokeless_user_current_smoker', 'current_smokeless_user', 'health')
dtypes: float64(402), object(27)
memory usage: 6.1+ MB


In [265]:
demographics_df = stages_mi_sub_df.xs('demographics', level='subset', axis=1)
demographics_df.head()

descriptive,participants_year_of_birth,participants_age,participants_sex,height_in_feet,height_in_inches,weight_in_pounds,body_mass_index_(bmi),participants_ethnicity_(hispanic_or_latino),participants_ethnicity_(sub_hispanic_or_latino_origin),participants_race_(main),...,do_you_work_a_split_shift,days_per_week_at_work,self-reported_work_start_time_current_shift,self-reported_work_end_time_current_shift,self-reported_work_start_time_next_shift,self-reported_work_end_time_next_shift,self-reported_work_start_time_3rd_shift,self-reported_work_start_time_no_3rd_shift,self-reported_work_end_time_3rd_shift,how_often_change_work_shifts
original,dem_0100,modified_dem_0110,dem_0500,dem_0600,dem_0610,dem_0700,dem_0800,dem_0900,dem_0910,dem_1000,...,sched_0510,sched_0600,sched_0700,sched_0800,sched_1100,sched_1200,sched_1500,sched_1501,sched_1600,sched_2100
0,1960.0,58.0,F,5.0,2.0,168.0,30.7,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1987.0,30.0,F,5.0,7.0,188.0,29.4,0.0,NaN,1.0,...,0.0,NaN,08:00:00,17:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,1988.0,30.0,F,5.0,7.0,165.0,25.8,0.0,NaN,1.0,...,0.0,3.0,07:00:00,19:00:00,07:00:00,19:00:00,NaN,1.0,NaN,1.0
3,1976.0,42.0,M,5.0,4.0,156.0,26.8,0.0,NaN,1.0,...,0.0,5.0,06:00:00,08:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,1982.0,36.0,M,5.0,3.0,255.0,45.2,0.0,NaN,1.0,...,0.0,5.0,08:15:00,17:30:00,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1860 entries, 0 to 1880
Data columns (total 35 columns):
 #   Column                                                                              Non-Null Count  Dtype  
---  ------                                                                              --------------  -----  
 0   (participants_year_of_birth, dem_0100)                                              1859 non-null   float64
 1   (participants_age, modified_dem_0110)                                               1859 non-null   float64
 2   (participants_sex, dem_0500)                                                        1859 non-null   object 
 3   (height_in_feet, dem_0600)                                                          1859 non-null   float64
 4   (height_in_inches, dem_0610)                                                        1859 non-null   float64
 5   (weight_in_pounds, dem_0700)                                                        1859 non-null   fl

In [ ]:
# cleaning steps in data wrangler
# there are 6 rows with the exact same demographics, whatever that means.